In [407]:
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import numpy as np
import json
import sqlite3
import datetime
from dateutil.relativedelta import relativedelta


SEARCH_WORD = "EXA FIRST"

DB_PATH = r"C:\python\dataOnline\anaslo_02\db\anaslo_02.db"

spreadSheet_ids = {
    "EXA FIRST": "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8",
    "アスカ狭山店": "179nJF0NvLng7xPKsd_NX2pJBXsDNsO8SJhOvUAvFk2I",
    "パールショップともえ川越店": "1i70joJ27Hs7inS-D89z9YMSJO1aRvaBeeWn0n9xpktY",
    }

# 検索キーワードよりホール名取得
SPREADSHEET_ID = spreadSheet_ids[SEARCH_WORD]

# スプレッドシート認証設定
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
jsonf = r"C:\python\dataOnline\anaslo_02\json\spreeadsheet-347321-ff675ab5ccbd.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
client = gspread.authorize(creds)
spreadsheet = client.open_by_key(SPREADSHEET_ID)

# # Table name 取得
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
tables = cursor.fetchall()
# print(tables)

cursor.execute(
    "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + SEARCH_WORD + "%",)
)
results = cursor.fetchall()

# 結果表示
if results:
    for hall_id, hall_name in results:
        # print(f" - hall_name: {hall_name}, hall_id: {hall_id}")
        print(f"🔍 '{SEARCH_WORD}' を含むホール名が見つかりました。")
else:
    print(f"❌ '{SEARCH_WORD}' を含むホール名は見つかりませんでした。")

query = """
-- 出玉データにホール名と機種名を結合して取得
SELECT
    r.*, 
    h.name AS hall_name,     -- ホール名を追加
    m.name AS model_name     -- 機種名を追加
FROM results r
JOIN halls h ON r.hall_id = h.hall_id  -- ホールと結合
JOIN models m ON r.model_id = m.model_id  -- 機種と結合
WHERE h.name = ?  -- 指定ホールのみ
AND m.name LIKE '%ジャグラー%'  -- ジャグラー系機種に限定
ORDER BY r.date DESC, r.unit_no ASC;
"""

df = pd.read_sql_query(query, conn, params=(hall_name,))
conn.close()

# ブドウシミュレーター
def grape_calculator_myfive(game, bb, rb, medals, cherry=True):
    bb_medals = 239.25
    rb_medals = 95.25
    replay_rate = 0.411
    if cherry:
        cherry_rate_high = 0.04228
    else:
        cherry_rate_high = 0.05847
    denominator_inner = (-medals - (game*3 - (bb*bb_medals + rb*rb_medals + game*replay_rate + game*cherry_rate_high))) / 8
    grape_rate = (game / denominator_inner) - ((game / denominator_inner) * 2)
    
    return grape_rate

def assign_area(unit_no, json_file_path):
    with open(json_file_path, "r", encoding="utf-8") as f:
        area_map = json.load(f)
    for rule in area_map:
        if rule["start"] <= unit_no <= rule["end"]:
            return rule["area"]
    return "その他"

df["date"] = pd.to_datetime(df["date"])
df.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
df = df[["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]]
df["BB_rate"] = (df["game"] / df["BB"]).round(1)
df["RB_rate"] = (df["game"] / df["RB"]).round(1)
df["Grape_rate"] = grape_calculator_myfive(df["game"], df["BB"], df["RB"], df["medals"], cherry=True).round(2)
df["Total_rate"] = (df["game"] / (df["BB"] + df["RB"])).round(1)
df["month"] = df["date"].dt.strftime("%Y-%m")
df["day"] = df["date"].dt.day
df["weekday"] = df["date"].dt.weekday
df["unit_last"] = df["unit_no"].astype(str).str[-1]

JSON_FILE_PATH = r"C:\python\dataOnline\anaslo_02\json\exa_area_map.json"
df["area"] = df["unit_no"].apply(lambda x: assign_area(x, JSON_FILE_PATH))

df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0)

print(f'データサイズ: {df.shape[0]} x {df.shape[1]}')
print(f'以下の日付のデータが含まれています')
print(df.date.unique()[0])
model_list = df["model_name"].unique()
print(f'以下のモデルが含まれています')
for i, model in enumerate(model_list):
    print(f'{i}: {model}', end=", ")
    
df.head()

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 58520 x 17
以下の日付のデータが含まれています
2025-04-23 00:00:00
以下のモデルが含まれています
0: ゴーゴージャグラー3, 1: マイジャグラーV, 2: ファンキージャグラー2, 3: アイムジャグラーEX-TP, 4: ミスタージャグラー, 5: ウルトラミラクルジャグラー, 6: ハッピージャグラーVIII, 7: ジャグラーガールズ, 8: SミスタージャグラーKK, 

,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,Grape_rate,Total_rate,month,day,weekday,unit_last,area
0,EXA FIRST,2025-04-23,ゴーゴージャグラー3,1001,2593,5,4,-1606,518.6,648.2,6.06,288.1,2025-04,23,2,1,a
1,EXA FIRST,2025-04-23,ゴーゴージャグラー3,1002,3106,16,7,853,194.1,443.7,5.82,135.0,2025-04,23,2,2,a
2,EXA FIRST,2025-04-23,ゴーゴージャグラー3,1003,1389,4,5,-315,347.2,277.8,6.21,154.3,2025-04,23,2,3,a
3,EXA FIRST,2025-04-23,ゴーゴージャグラー3,1004,1918,9,5,168,213.1,383.6,6.33,137.0,2025-04,23,2,4,a
4,EXA FIRST,2025-04-23,ゴーゴージャグラー3,1005,3798,9,15,-947,422.0,253.2,5.91,158.2,2025-04,23,2,5,a


## 過去7日間の差枚と翌日の結果を比較

In [408]:
# model_name = model_list[1]
# today = datetime.date.today()
# target_date = today - datetime.timedelta(days=1)
# start = today - relativedelta(days=1)
# end = today - relativedelta(days=30)
# unit_no = 1021

# df_tmp = df.copy()
# df_tmp = df_tmp[(df_tmp["model_name"] == model_name)]
# df_tmp = df_tmp[(df_tmp["date"].dt.date <= start) & (df_tmp["date"].dt.date >= end)]
# df_tmp = df_tmp[(df_tmp["unit_no"] >= unit_no)]

# medals = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="medals", aggfunc="sum",)
# game = df_tmp.pivot_table(index=["model_name", "unit_no", ], columns="date", values="game", aggfunc="sum",)
# bb = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="BB", aggfunc="sum",)
# rb = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="RB", aggfunc="sum",)
# rb_rate = (game / rb).round(1)
# total_rate = (game / (bb+rb)).round(1)

# medals = medals.iloc[:, 7:].iloc[:, ::-1]
# rb_rate = rb_rate.iloc[:, 7:].iloc[:, ::-1]
# total_rate = total_rate.iloc[:, 7:].iloc[:, ::-1]

# rolling_7d_sum = medals.iloc[:, ::-1].rolling(window=7, axis=1, min_periods=1).sum().iloc[:, ::-1].iloc[:, :-6]
# rolling_7d_sum.columns = [f"{col.strftime('%y-%m-%d')}_7d_sum" for col in rolling_7d_sum.columns]
# # rolling_7d_sum = rolling_7d_sum.iloc[:, ::-1]

# rolling_7d_rank = rolling_7d_sum.rank(method="min", ascending=True).astype(int)
# rolling_7d_rank.columns = [c.replace("sum", "rank") for c in rolling_7d_rank.columns]

# medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
# rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
# total_rate.columns = pd.MultiIndex.from_product([["TOTAL_RATE"], total_rate.columns])
# rolling_7d_sum.columns = pd.MultiIndex.from_product([["7D_sum"], rolling_7d_sum.columns])
# rolling_7d_rank.columns = pd.MultiIndex.from_product([["RANK"], rolling_7d_rank.columns])

# print(rolling_7d_sum.shape)
# print(rolling_7d_rank.shape)
# print(rb_rate.shape)
# print(total_rate.shape)
# display(rolling_7d_sum.head(2))
# display(rolling_7d_rank.head(2))
# display(rb_rate.head(2))
# display(total_rate.head(2))


In [ ]:
# # 同じ列数を前提（もしくは短い方に合わせる）
# columns_interleaved = [
#     col for pair in zip(rolling_7d_rank.columns, rolling_7d_sum.columns, medals.columns, rb_rate.columns, total_rate.columns) for col in pair
# ]
# merged = pd.concat([rolling_7d_rank, rolling_7d_sum, medals, rb_rate, total_rate], axis=1)[columns_interleaved]
# # 空行を追加
# empty_index = pd.MultiIndex.from_tuples([("", "")], names=merged.index.names)
# merged_with_blanks = pd.DataFrame(
#     [[""] * merged.shape[1]], columns=merged.columns, index=empty_index
# )
# empty_row = merged_with_blanks.copy()
# blocks = [
#     merged.iloc[0:6],
#     merged.iloc[6:12],
#     merged.iloc[12:21],
#     merged.iloc[21:30],
#     merged.iloc[30:46],
#     merged.iloc[46:54],
#     merged.iloc[54:61],
# ]
# for block in blocks:
#     merged_with_blanks = pd.concat([merged_with_blanks, block, empty_row])

# # rank = rolling_7d_sum.iloc[:, 1].rank(method="min", ascending=True).astype(int)
# merged_with_blanks.to_csv(f"{model_name}_merged.csv", encoding="utf_8_sig")
# merged_with_blanks.head()

RANK          7D_sum              MEDALS  \
                   25-04-22_7d_rank 25-04-22_7d_sum 2025-04-22 00:00:00   
model_name unit_no                                                        
                                                                          
マイジャグラーV   1021                  20         -1225.0                -374   
           1022                  47          1123.0                1112   
           1023                  37          -116.0                -162   
           1024                  48          1149.0                 479   

                               RB_RATE          TOTAL_RATE             RANK  \
                   2025-04-22 00:00:00 2025-04-22 00:00:00 25-04-21_7d_rank   
model_name unit_no                                                            
                                                                              
マイジャグラーV   1021                  917.0               305.7               45   
           1022                  418.9               142.8               35   
           1023                  301.8               156.5               17   
           1024                  407.2               135.8               37   

                            7D_sum              MEDALS             RB_RATE  \
                   25-04-21_7d_sum 2025-04-21 00:00:00 2025-04-21 00:00:00   
model_name unit_no                                                           
                                                                             
マイジャグラーV   1021              940.0                 724               405.9   
           1022              135.0                -462               440.0   
           1023            -1419.0                -762               467.2   
           1024              202.0                2515               309.9   

                            TOTAL_RATE  ...             RANK          7D_sum  \
                   2025-04-21 00:00:00  ... 25-04-07_7d_rank 25-04-07_7d_sum   
model_name unit_no                      ...                                    
                                        ...                                    
マイジャグラーV   1021                  142.0  ...               32           381.0   
           1022                  180.0  ...               29          -356.0   
           1023                  233.6  ...               22         -1206.0   
           1024                  121.9  ...               14         -2200.0   

                                MEDALS             RB_RATE  \
                   2025-04-07 00:00:00 2025-04-07 00:00:00   
model_name unit_no                                           
                                                             
マイジャグラーV   1021                   1618               253.3   
           1022                   1615               261.3   
           1023                   -685               336.0   
           1024                    391               309.9   

                            TOTAL_RATE             RANK          7D_sum  \
                   2025-04-07 00:00:00 25-04-06_7d_rank 25-04-06_7d_sum   
model_name unit_no                                                        
                                                                          
マイジャグラーV   1021                  104.8               17         -1469.0   
           1022                  120.6               12         -2633.0   
           1023                  252.0               16         -1774.0   
           1024                  146.8               13         -2573.0   

                                MEDALS             RB_RATE          TOTAL_RATE  
                   2025-04-06 00:00:00 2025-04-06 00:00:00 2025-04-06 00:00:00  
model_name unit_no                                                              
                                                                                
マイジャグラーV   1021                   -579               333.8               161.5  
           1022             

## コード確認テスト

In [411]:
def extract_and_merge_model_data(df, model_name, start_date, end_date):
    # 対象期間・モデルのデータを抽出
    df_filtered = df.copy()
    df_filtered = df_filtered[(df_filtered["date"].dt.date <= start_date) & (df_filtered["date"].dt.date >= end_date)]
    df_filtered = df_filtered[(df_filtered["model_name"] == model_name)]
    
    # 各種ピボットテーブル
    medals = df_filtered.pivot_table(index=["area", "model_name", "unit_no"], columns="date", values="medals", aggfunc="sum")
    game = df_filtered.pivot_table(index=["area", "model_name", "unit_no"], columns="date", values="game", aggfunc="sum",)
    rb_rate = df_filtered.pivot_table(index=["area", "model_name", "unit_no"], columns="date", values="RB_rate", aggfunc="sum")
    total_rate = df_filtered.pivot_table(index=["area", "model_name", "unit_no"], columns="date", values="Total_rate", aggfunc="sum")
    grape_rate = df_filtered.pivot_table(index=["area", "model_name", "unit_no"], columns="date", values="Grape_rate", aggfunc="sum")
    
    # 日付列を反転・スライス
    medals = medals.iloc[:, 7:].iloc[:, ::-1]
    game = game.iloc[:, 7:].iloc[:, ::-1]
    rb_rate = rb_rate.iloc[:, 7:].iloc[:, ::-1]
    total_rate = total_rate.iloc[:, 7:].iloc[:, ::-1]
    grape_rate = grape_rate.iloc[:, 7:].iloc[:, ::-1]

    # 7日間累積とランク
    rolling_7d_sum = medals.iloc[:, ::-1].rolling(window=7, min_periods=1).sum().iloc[:, ::-1].iloc[:, :-6]
    rolling_7d_sum.columns = [f"{col.strftime('%y-%m-%d')}_7d_sum" for col in rolling_7d_sum.columns]
    rolling_7d_sum = rolling_7d_sum.iloc[:, ::-1]
    rolling_7d_rank = rolling_7d_sum.rank(method="min", ascending=True).fillna(0).replace([np.inf, -np.inf], 0).astype(int)
    rolling_7d_rank.columns = [c.replace("sum", "rank") for c in rolling_7d_rank.columns]
    rolling_7d_sum = rolling_7d_sum.iloc[:, ::-1]
    rolling_7d_rank = rolling_7d_rank.iloc[:, ::-1]

    # MultiIndex化（ラベル付け）
    medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
    game.columns = pd.MultiIndex.from_product([["GAME"], game.columns])
    rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
    total_rate.columns = pd.MultiIndex.from_product([["TOTAL_RATE"], total_rate.columns])
    grape_rate.columns = pd.MultiIndex.from_product([["GRAPE_RATE"], grape_rate.columns])
    rolling_7d_sum.columns = pd.MultiIndex.from_product([["7D_sum"], rolling_7d_sum.columns])
    rolling_7d_rank.columns = pd.MultiIndex.from_product([["RANK"], rolling_7d_rank.columns])

    # 列を交互に整列して統合・NaN除去
    interleaved_cols  = [
        col for pair in zip(
            rolling_7d_rank.columns,
            rolling_7d_sum.columns,
            medals.columns,
            game.columns,
            rb_rate.columns,
            total_rate.columns,
            grape_rate.columns
            ) for col in pair
        ]
    merged = pd.concat([
        rolling_7d_rank, rolling_7d_sum, medals, game, rb_rate, total_rate, grape_rate], 
                       axis=1
        )[interleaved_cols]
    merged = merged[~merged.iloc[:, 2].isna()]

    # エリアごとに空行挿入して整形
    merged_by_area = pd.DataFrame()
    for area in merged.index.get_level_values("area").unique():
        area_merged = merged.xs(area, level="area", drop_level=False)
        if not area_merged.empty:
            empty_index = pd.MultiIndex.from_tuples([("", " ", " ")], names=merged.index.names)
            empty_row = pd.DataFrame([[""] * area_merged.shape[1]], index=empty_index, columns=area_merged.columns)
            merged_by_area = pd.concat([merged_by_area, area_merged, empty_row])
    # インデックス削除
    merged_by_area = merged_by_area.droplevel("area")
    
    return merged_by_area


today = datetime.date.today()
start_date = today - relativedelta(days=1)
end_date = start_date - relativedelta(days=30)

merged_by_model = pd.DataFrame()
model_list = ['マイジャグラーV', 'アイムジャグラーEX-TP', 'ゴーゴージャグラー3', 'ファンキージャグラー2', 'ミスタージャグラー' ]
for model in model_list:
    merged_by_area = extract_and_merge_model_data(df, model, start_date, end_date)
    # モデル間の区切り用空行追加して結合
    empty_index = pd.MultiIndex.from_tuples([(" ", " ")], names=merged_by_area.index.names)
    empty_row = pd.DataFrame([[""] * merged_by_area.shape[1]], index=empty_index, columns=merged_by_area.columns)
    merged_by_model = pd.concat([merged_by_model, merged_by_area, empty_row], axis=0)
# 出力
merged_by_model.to_csv(f"merged_by_model_{start_date}_{end_date}.csv", encoding="utf_8_sig")
display(merged_by_model)

RANK          7D_sum              MEDALS  \
                   25-04-23_7d_rank 25-04-23_7d_sum 2025-04-23 00:00:00   
model_name unit_no                                                        
マイジャグラーV   1021                  29          -191.0              -191.0   
           1022                  23          -382.0              -191.0   
           1023                  15         -1032.0              -650.0   
           1024                  35           221.0              1253.0   
           1025                  30          -105.0              -326.0   
...                             ...             ...                 ...   
ミスタージャグラー  1111                   5          1815.0              -268.0   
           1112                   4          1641.0              -174.0   
           1113                   3          1576.0               -65.0   
                                                                          
                                                                          

                                  GAME             RB_RATE  \
                   2025-04-23 00:00:00 2025-04-23 00:00:00   
model_name unit_no                                           
マイジャグラーV   1021                 2325.0               465.0   
           1022                 1446.0               723.0   
           1023                 1432.0              1432.0   
           1024                 2857.0               219.8   
           1025                 1046.0               348.7   
...                                ...                 ...   
ミスタージャグラー  1111                  900.0               900.0   
           1112                  261.0               261.0   
           1113                 6391.0               266.3   
                                                             
                                                             

                            TOTAL_RATE          GRAPE_RATE             RANK  \
                   2025-04-23 00:00:00 2025-04-23 00:00:00 25-04-22_7d_rank   
model_name unit_no                                                            
マイジャグラーV   1021                  178.8                5.57               22   
           1022                  180.8                 6.2               30   
           1023                  238.7                6.72               28   
           1024                  105.8                 5.8               35   
           1025                  174.3                6.27               34   
...                                ...                 ...              ...   
ミスタージャグラー  1111                  225.0                5.95                5   
           1112                  261.0                5.28                6   
           1113                  138.9                 5.9                1   
                                                                              
                                                                              

                            7D_sum              MEDALS  ...  \
                   25-04-22_7d_sum 2025-04-22 00:00:00  ...   
model_name unit_no                                      ...   
マイジャグラーV   1021             -374.0              -374.0  ...   
           1022              738.0              1112.0  ...   
           1023              576.0              -162.0  ...   
           1024             1055.0               479.0  ...   
           1025              946.0              -109.0  ...   
...                            ...                 ...  ...   
ミスタージャグラー  1111              317.0               438.0  ...   
           1112              326.0                 9.0  ...   
           1113             -348.0              -674.0  ...   
                                                        ...   
                                                        ...   

                               RB_RATE          TOTAL_RATE  \
                   2025-04-07 00:00:00 2025-04-07 00:00:00 

## 2024年のGWの分析

In [406]:
# model_name = model_list[1]
today = datetime.date.today()
target_date = today - datetime.timedelta(days=1)
start_date = datetime.date(2024, 5, 5)
end_date = datetime.date(2024, 4, 27)

merged_by_model = pd.DataFrame()
model_list = ['マイジャグラーV', 'アイムジャグラーEX-TP', 'ゴーゴージャグラー3', 'ファンキージャグラー2', 'ミスタージャグラー' ]
for model in model_list:
    df_filtered = df.copy()
    df_filtered = df_filtered[(df_filtered["date"].dt.date <= start_date) & (df_filtered["date"].dt.date >= end_date)]
    df_filtered = df_filtered[(df_filtered["model_name"] == model)]